In [105]:
import pandas as pd 
import numpy as np
import networkx as nx
import re
import swifter
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df_users = pd.read_csv("data/combined_twitter_data_with_tweets_corpus.csv") #id

# get tweets df
filenames_tweets = [
    "data/all tweets 2017/tweets_fake_followers.csv",
    "data/all tweets 2017/tweets_genuine_accounts.csv",
    "data/all tweets 2017/tweets_social_spambots_1.csv",
    "data/all tweets 2017/tweets_social_spambots_2.csv",
    "data/all tweets 2017/tweets_social_spambots_3.csv",
    "data/all tweets 2017/tweets_traditional_spambots_1.csv",

    "data/tweets 2015/tweets_E13.csv",
    "data/tweets 2015/tweets_FSF.csv",
    "data/tweets 2015/tweets_INT.csv",
    "data/tweets 2015/tweets_TFP.csv",
    "data/tweets 2015/tweets_TWT.csv"
]

for i,fn in enumerate(filenames_tweets):
    if i == 0:
        df_tweets = pd.read_csv(fn, encoding='ISO-8859-1')
    else:
        df_tweets = pd.concat([df_tweets, pd.read_csv(fn, encoding='ISO-8859-1') ], axis=0) #user_id

/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_11877/2074397400.py:21: DtypeWarning: Columns (8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tweets = pd.read_csv(fn, encoding='ISO-8859-1')
/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_11877/2074397400.py:23: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tweets = pd.concat([df_tweets, pd.read_csv(fn, encoding='ISO-8859-1') ], axis=0) #user_id
/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_11877/2074397400.py:23: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tweets = pd.concat([df_tweets, pd.read_csv(fn, encoding='ISO-8859-1') ], axis=0) #user_id
/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_11877/2074397400.py:23: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tweets = pd.con

## Build edge list
- mentions 
- following 

In [140]:
edges = set()

def add_edges(user_id, tweet):
    mentioned_users = re.findall(r"@[a-zA-Z0-9_]{0,15}", str(tweet))
    if mentioned_users:
        mentioned_users = list(map(lambda x: x[1:], mentioned_users)) #remove @ in name
        for mentioned in mentioned_users: #get the ID from df_users 
            mentioned_id = df_users[df_users['screen_name'] == mentioned]
            if mentioned_id.size > 0:
                edges.add((user_id, mentioned_id.loc[:,"id"].values[0])) #add edge 
            else: #mentioned_id not in df_users 
                continue
    else: #no mentions in tweet
        return
    return

In [141]:
df_tweets.swifter.apply(lambda x: add_edges(x['user_id'], x['text']), axis=1)

Pandas Apply:   0%|          | 0/9465373 [00:00<?, ?it/s]

0         None
1         None
2         None
3         None
4         None
          ... 
114187    None
114188    None
114189    None
114190    None
114191    None
Length: 9465373, dtype: object

## Implement Reciprocity

In [142]:
edge_list = list(edges)
directed_graph = nx.DiGraph()
directed_graph.add_edges_from(edge_list)
#plt.figure(figsize=(10,10))
#nx.draw_circular(directed_graph, with_labels=True, node_size=1800, alpha=0.6), font_size=20)

# Reciprocity (Directed Graph)
# The ratio of the number of relations which are reciprocated over the total number of relations in a directed network
# real people could have high level of reciprocity in the network 
nx.algorithms.reciprocity(directed_graph)

0.05016648168701443